If the video has an available transcript, retrieves the full transcript text using youtube_transcript_api,
otherwise, returns the raised exception if no transcript available

In [28]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs

In [34]:
video_url = "https://www.youtube.com/watch?v=GuM6dQGRFyQ"
video_url = "https://www.youtube.com/watch?v=-hkEzwaY754"

In [39]:
def get_youtube_video_id(url: str) -> str | None:
    parsed = urlparse(url)

    # youtu.be/<id>
    if parsed.netloc in ("youtu.be", "www.youtu.be"):
        return parsed.path.lstrip("/")

    # youtube.com/*
    if "youtube.com" in parsed.netloc:
        # watch?v=<id>
        if parsed.path == "/watch":
            return parse_qs(parsed.query).get("v", [None])[0]

        # /embed/<id>, /shorts/<id>
        parts = parsed.path.split("/")
        if len(parts) >= 3 and parts[1] in {"embed", "shorts"}:
            return parts[2]

    return None
    
def retrieve_video_transcript(video_url:str) -> str:
    ytt_api = YouTubeTranscriptApi()
    full_text=""
    try : 
        fetched_transcript = ytt_api.fetch(get_youtube_video_id(video_url))
    except Exception as e:
        print(f"no transcript available for {video_url}")
        return("")
    for snippet in fetched_transcript.snippets:
        full_text+= snippet.text + " "
    return(full_text)

In [40]:
test_text = retrieve_video_transcript(video_url)

no transcript available for https://www.youtube.com/watch?v=-hkEzwaY754


In [41]:
test_text

''